In [32]:
import numpy as np
import random

import torch
import torch.nn as nn
import torchvision.utils as vutils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import matplotlib.pylab as plt

import glob
from glob import glob
import nibabel as nb

import gzip
import os

import cv2

In [17]:
# MriFiles = glob(r"C:\Users\Xiaowei\Documents\MRI_Vols\*\*T1w.npy.gz")
# SosFiles = glob(r"C:\Users\Xiaowei\Documents\MRI_Vols\*\*Vp.npy.gz")

# images = []
# for i in range(len(MriFiles)):
#     f1 = gzip.GzipFile(MriFiles[i], "r") # file type is now gzip.GzipFile
#     data1 = np.load(f1)
#     img1 = nb.Nifti1Image(data1,None)
#     img1.header.get_xyzt_units()
#     img1.to_filename(os.path.join('build', MriFiles[i].replace('.npy','.nii')))  # Save as NiBabel file

#     f2 = gzip.GzipFile(SosFiles[i], "r") # file type is now gzip.GzipFile
#     data2 = np.load(f2)
#     img2 = nb.Nifti1Image(data2,None)
#     img2.header.get_xyzt_units()
#     img2.to_filename(os.path.join('build', SosFiles[i].replace('.npy','.nii')))  # Save as NiBabel file

In [18]:
mri_files = glob(r"C:\Users\Xiaowei\Documents\MRI_Vols\*\*_T1w.nii.gz")
sos_files = glob(r"C:\Users\Xiaowei\Documents\MRI_Vols\*\Vp.nii.gz")
print("Number of MRIs: "+str(len(mri_files)))
print("Number of SoS images: "+str(len(sos_files)))

Number of MRIs: 5
Number of SoS images: 5


In [19]:
def create_sections(mri_im, sos_im, labels):
  [slice,y0,y1,x0,x1] = labels.split(',')
  temp_m = np.array(mri_im[:,:,int(slice)][int(y0):int(y1),int(x0):int(x1)])
  temp_s = np.array(sos_im[:,:,int(slice)][int(y0):int(y1),int(x0):int(x1)])

  return [temp_m, temp_s]

In [21]:
sections = []
brain_sections = []
size = 64
for f in range(len(mri_files)):
    mri_data = nb.load(mri_files[f]).get_fdata()
    sos_data = nb.load(sos_files[f]).get_fdata()

    # extracting regions of whole image
    n=0
    slice = [i for i in range(130,220)]
    for s in slice:
        for n in range(0,int(mri_data.shape[0]/size)):
            for k in range(0,int(mri_data.shape[2]/size)):
                sections.append(create_sections(mri_data,sos_data,str(s)+","+str(n*size)+","+str(n*size+size)+','+str(k*size)+","+str(k*size+size)))
            
# extracting the brain patches
for i in range(len(sections)):
    if(i%5==0 or (i-4)==5): continue
    else: brain_sections.append(sections[i])

print("Full dataset: " +str(np.array(sections).shape))
print("Brain-containing images: " +str(len(brain_sections)))

Full dataset: (9000, 2, 64, 64)
Brain-containing images: 7199


In [38]:
for i in range(len(brain_sections)):
    cv2.imwrite(r'C:\Users\Xiaowei\Desktop\Clara\CycleGAN\mri2sos_dataset\A\mri_img'+str(i)+'.jpg', brain_sections[i][0])
    cv2.imwrite(r'C:\Users\Xiaowei\Desktop\Clara\CycleGAN\mri2sos_dataset\B\sos_img'+str(i)+'.jpg', brain_sections[i][1])